In [5]:
from architectures_v1 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score

import os
import numpy as np
import tensorflow as tf
import pandas as pd
import time
import random

## Loading Training/Testing/Validation Data

In [6]:
dataset = "PenDigits"

x_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_train.npz')))
x_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_test.npz')))

x_train = x_train_load['arr_0']
x_test = x_test_load['arr_0']

x_all = np.concatenate((x_train, x_test), axis = 0)

n_instances = x_all.shape[0]
print(f"Total Instances: {n_instances}")

y_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_train.npz')))
y_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_test.npz')))

y_train = np.asarray(y_train_load['arr_0'], dtype = np.uint64)
y_test = np.asarray(y_test_load['arr_0'], dtype = np.uint64)

y_all = np.concatenate((y_train, y_test), axis = 0)

n_validation = int(0.1*n_instances)
print(f"Validation Instances: {n_validation}")

ind_validation = random.sample(range(0, n_instances), n_validation)
x_valid = x_all[ind_validation, :, :]
y_valid = y_all[ind_validation]

x_all = np.delete(x_all, ind_validation, axis = 0)
y_all = np.delete(y_all, ind_validation, axis = 0)

print(f"x_all: {x_all.shape} - y_all: {y_all.shape}")
print(f"x_valid: {x_valid.shape} - y_valid: {y_valid.shape}")

Total Instances: 10992
Validation Instances: 1099
x_all: (9893, 8, 2) - y_all: (9893,)
x_valid: (1099, 8, 2) - y_valid: (1099,)


In [7]:
seed = 42
tf.random.set_random_seed(seed)
np.random.seed(seed)

length = x_all.shape[1]
n_features = x_all.shape[-1]

model = Sequential()

model.add(RNN(OrthogonalCell(units = 212,
                             order = 256,
                             variant = 'ct_laguerre',
                             dt = 1,
                            input_dims = n_features), 
              input_shape = (length, n_features),
             return_sequences = False))
model.add(Dense(to_categorical(y_all).shape[-1], activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn_1 (RNN)                  (None, 212)               166418    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2130      
Total params: 168,548
Trainable params: 102,496
Non-trainable params: 66,052
_________________________________________________________________


In [8]:
if not os.path.exists(os.path.abspath(os.path.join('models', dataset))):
    os.mkdir(os.path.abspath(os.path.join('models', dataset)))

kf = KFold(n_splits = 5)
sdk = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    
n_fold = 1

acc_per_fold = []
epoch_per_fold = []
loss_per_fold = []
rec_per_fold = []
prec_per_fold = []
f1_per_fold = []

for train, test in sdk.split(x_all, y_all):
    
    file_path = os.path.abspath(os.path.join('models', dataset, f'{dataset}-Laguerre_{n_fold}.hdf5'))

    callbacks = [
        ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only = True, mode = 'min', verbose = 1), 
        EarlyStopping(monitor = 'val_loss', patience = 50, mode = 'min')]

    t = time.time()
    
    print(f"Fold: {n_fold}\n")
    result = model.fit(x_all[train], 
                       to_categorical(y_all[train]),
                       epochs = 2000, 
                       batch_size = 100, 
                       validation_data = (x_valid, to_categorical(y_valid)), 
                       callbacks = callbacks)

    print(f"Training time: {time.time() - t} s")

    df_results = pd.DataFrame(result.history)
    df_results.to_csv(os.path.abspath(os.path.join('models', dataset, f'Laguerre_results_{n_fold}.csv')))
    
    model.load_weights(file_path)
    scores = model.evaluate(x_all[test], to_categorical(y_all[test]))
    loss_per_fold.append(scores[0])
    acc_per_fold.append(scores[1])
    epoch_per_fold.append(np.argmin(result.history['val_loss']))
    
    # Computing predictions
    y_pred_test = np.argmax(model.predict(x_all[test]), axis = 1)
    
    f1_per_fold.append(f1_score(y_all[test], y_pred_test, average = 'macro'))
    rec_per_fold.append(recall_score(y_all[test], y_pred_test, average = 'macro'))
    prec_per_fold.append(precision_score(y_all[test], y_pred_test, average = 'macro'))
    
    n_fold += 1
    
df_scores = pd.DataFrame(columns = ['F1', 'Loss', 'Accuracy', 'Precision', 'Recall'])
df_scores['F1'] = f1_per_fold
df_scores['Loss'] = loss_per_fold
df_scores['Accuracy'] = acc_per_fold
df_scores['Precision'] = prec_per_fold
df_scores['Recall'] = rec_per_fold

df_scores.to_csv(os.path.abspath(os.path.join('models', dataset, f'Laguerre_results_k-Folds.csv')))

Fold: 1

Train on 7914 samples, validate on 1099 samples
Epoch 1/2000
7700/7914 [============================>.] - ETA: 0s - loss: 1.0784 - acc: 0.6521
Epoch 00001: val_loss improved from inf to 0.79744, saving model to /data/home/dorads/Documents/GitHub_Repositories/01_Maintained/NeuralODE/models/PenDigits/PenDigits-Laguerre_1.hdf5
7914/7914 [==============================] - 1s 163us/sample - loss: 1.0711 - acc: 0.6549 - val_loss: 0.7974 - val_acc: 0.7580
Epoch 2/2000
7600/7914 [===========================>..] - ETA: 0s - loss: 0.6292 - acc: 0.7871
Epoch 00002: val_loss improved from 0.79744 to 0.62075, saving model to /data/home/dorads/Documents/GitHub_Repositories/01_Maintained/NeuralODE/models/PenDigits/PenDigits-Laguerre_1.hdf5
7914/7914 [==============================] - 1s 98us/sample - loss: 0.6260 - acc: 0.7885 - val_loss: 0.6208 - val_acc: 0.7962
Epoch 3/2000
7400/7914 [===========================>..] - ETA: 0s - loss: 0.5126 - acc: 0.8232
Epoch 00003: val_loss improved from